In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import numpy as np
import gc
from sklearn.utils import shuffle
warnings.filterwarnings('ignore')

# 数据读取

In [ ]:
import pandas as pd
from pathlib import Path

def transfer_data(df):
    df['生产线编号'] = (
        df['生产线编号']
        .str.replace("M", "", regex=False)
        .astype("int16")
    )
    return df

def load_data(folder, ids):
    data = {}
    for i in ids:
        path = Path(folder) / f"M{i}.csv"
        df = pd.read_csv(path)
        data[f"M{i}"] = transfer_data(df)
    return data

data_train = load_data("data/train", range(101, 111))
data_test = load_data("data/test", range(201, 203))


# 预处理+特征工程

In [ ]:
# 故障类别Lables-encoding
def update_machine_info(df):
    fault_code_map = {
        '物料推送装置故障1001': 1,
        '物料检测装置故障2001': 2,
        '填装装置检测故障4001': 3,
        '填装装置定位故障4002': 4,
        '填装装置填装故障4003': 5,
        '加盖装置定位故障5001': 6,
        '加盖装置加盖故障5002': 7,
        '拧盖装置定位故障6001': 8,
        '拧盖装置拧盖故障6002': 9
    }

    cols = list(fault_code_map.keys())
    codes = np.array(list(fault_code_map.values()), dtype=np.int8)

    fault_matrix = df[cols].values
    df['机器状态'] = (fault_matrix * codes).max(axis=1).astype('int8')

    df.drop(columns=cols, inplace=True)

    return df

In [ ]:
# 判断异常值（是否存在时间间断）
def judge_continuous(df):
    feature_cols = list(df.columns[:2]) + list(df.columns[3:-1])

    result = []

    for _, g in df.groupby('日期'):
        diff = g[feature_cols].diff().dropna()
        abnormal = diff[(diff.abs() > 1).any(axis=1)]
        result.append(abnormal)

    return pd.concat(result)

In [ ]:
# 特征简化
def feature_fusion(df):
    df['物料推送气缸状态'] = (df['物料推送气缸推送状态'] ^df['物料推送气缸收回状态']).astype('int8')
    df['填装定位器状态'] = (df['填装定位器固定状态'] ^df['填装定位器放开状态']).astype('int8')
    df.drop(
        columns=[
            '物料推送气缸推送状态',
            '物料推送气缸收回状态',
            '填装定位器固定状态',
            '填装定位器放开状态'
        ],
        inplace=True
    )
    return df

In [ ]:
# 构建新特征
def creat_feature(df):
    # 物料抓取差
    df['物料抓取差'] = df['物料待抓取数'] - df['物料抓取数']
    # 物料未加盖数
    df['物料未加盖数'] = (df['填装下降数'] - df['加盖检测数'] * 3) % 3
    # 拧盖剩余数
    df['拧盖剩余数'] = df['加盖下降数'] - df['拧盖检测数']
    # 装填完成步数
    df['装填完成步数'] = df['物料抓取数'] + df['填装旋转数'] - df['填装下降数'] * 2
    # 加盖完成步数
    df['加盖完成步数'] = df['加盖检测数'] + df['加盖定位数'] + df['推盖数'] - df['加盖下降数'] * 3
    # 拧盖完成步数
    df['拧盖完成步数'] = df['拧盖检测数'] + df['拧盖定位数'] + df['拧盖下降数'] + df['拧盖旋转数'] - df['拧盖数'] * 4
    return df

In [ ]:
# 状态持续时间 Duration 特征构建
feature_groups = [
    ['物料推送气缸状态'],
    ['物料推送数'],
    ['放置容器数','容器上传检测数'],
    ['物料待抓取数'],
    ['填装检测数'],
    ['填装定位器状态'],
    ['物料抓取数', '填装旋转数','填装下降数'],
    ['加盖检测数','加盖定位数'],
    ['推盖数', '加盖下降数'],
    ['拧盖检测数','拧盖定位数'],
    ['拧盖下降数', '拧盖旋转数']
]

def create_duration_feature(df, feature_groups):
    for idx, cols in enumerate(feature_groups):
        col_name = f"{idx}_Duration"

        #  判断状态是否发生变化
        change_flag = (df[cols] != df[cols].shift()).any(axis=1)
        # 对变化点进行分组编号
        group_id = change_flag.cumsum()
        # 计算每个 group 内的累计长度
        df[col_name] = (
            df.groupby(group_id).cumcount() + 1
        ).astype('int16')

    return df

In [ ]:
selected_columns = ['物料推送气缸状态', '物料推送数', '物料待抓取数', '放置容器数', '容器上传检测数','填装检测数', 
    '填装定位器状态', '物料抓取数', '填装旋转数', '填装下降数', '填装数', '加盖检测数', '加盖定位数',
    '推盖数', '加盖下降数', '加盖数', '拧盖检测数', '拧盖定位数', '拧盖下降数', '拧盖旋转数', '拧盖数']

# 删除所有列中的重复行，随机保留一行
def drop_duplicates_all(df,selected_columns):
    df_shuffle = shuffle(df, random_state=28)
    df_unique = df_shuffle.drop_duplicates(subset=selected_columns)
    df_unique = df_unique.sort_values(by=['日期','时间'], ascending=True)
    return df_unique

# 删除非故障列的重复行，随机保留一行 
def drop_duplicates(df,selected_columns):
    fault_data = df[df['机器状态']!=0]
    normal_data = df[df['机器状态']==0]
    normal_data = shuffle(normal_data, random_state=28)
    
    normal_data = normal_data.drop_duplicates(subset=selected_columns)
    
    df_unique = pd.concat([fault_data, normal_data])
    df_unique = df_unique.sort_values(by=['日期','时间'], ascending=True)
    return df_unique

In [ ]:
# 单条流水线完整处理
def process_machine_data(df, feature_groups, selected_columns, train):
    df = update_machine_info(df)
    feature_fusion(df)
    df = creat_feature(df)
    df = create_duration_feature(df, feature_groups)
    if train == True:
        df = drop_duplicates(df, selected_columns)
    return df

# 批量处理字典数据
def process_all_machines(data_dict, feature_groups, selected_columns, train=True):
    processed_dict = {}
    for machine_id, df in data_dict.items():
        df_processed = process_machine_data(df, feature_groups, selected_columns, train)
        processed_dict[machine_id] = df_processed
    return processed_dict


data_train_processed = process_all_machines(data_train, feature_groups, selected_columns, train=True)
data_test_processed = process_all_machines(data_test, feature_groups, selected_columns, train=False)


In [ ]:
import os

def save_processed_data(data_dict, folder):
    # 创建目录（如果不存在）
    os.makedirs(folder, exist_ok=True)

    for machine_id, df in data_dict.items():
        file_path = os.path.join(folder, f"{machine_id}.csv")
        df.to_csv(file_path, index=False)
        print(f"Saved {file_path}")

# 保存过程数据
save_processed_data(data_train_processed, "temp_data/train")
save_processed_data(data_test_processed, "temp_data/test")